# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [13]:
#pip install pandas

In [26]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Lano\ironhack\week7\Day3\Lab\lab-chains-in-langchain\data\Data.csv')

In [27]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain : Done

In [28]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [33]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7, api_key=OPENAI_API_KEY)  # كلما قلت القيمة أصبح النص أكثر دقة وأقل إبداعًا


In [34]:
prompt = ChatPromptTemplate.from_template(
    "Write a detailed and engaging description for the following product:\nProduct name: {product}\nDescription:"
)
#Write a query that would take a variable to describe any product
 

In [38]:

chain = LLMChain(llm=llm, prompt=prompt)

product = df["Product"].iloc[0]  # نختار أول منتج من البيانات
response = chain.run({"product": product})
print(response)


C:\Users\Lano\AppData\Local\Temp\ipykernel_23268\3866517586.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"product": product})


Indulge in luxurious comfort with our Queen Size Sheet Set. Crafted from premium quality, ultra-soft microfiber fabric, these sheets are sure to provide you with a blissful night's sleep. The set includes a flat sheet, fitted sheet, and two pillowcases, all designed to fit a standard queen size bed perfectly.

The breathable and lightweight material ensures a cool and comfortable sleeping experience, while the durable construction guarantees long-lasting use. The sheets are wrinkle-resistant and easy to care for, making them a convenient addition to any bedroom.

Available in a variety of colors and patterns, our Queen Size Sheet Set allows you to customize your bedding to suit your personal style. Whether you prefer classic neutrals or vibrant hues, there is a design to complement any decor.

Transform your bedroom into a peaceful retreat with our Queen Size Sheet Set. Treat yourself to the ultimate in comfort and style, and elevate your sleeping experience to new heights.


In [37]:
# product = #Select a product type to be describe
# chain.run(product)

## SimpleSequentialChain

In [39]:
from langchain.chains import SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [40]:
# LLM model
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
# Prompt 1: Generate a creative product name
first_prompt = ChatPromptTemplate.from_template(
    "Generate a catchy and creative product name for the following item:\n{product}"
) #Repeat the initial query or create a new query that would feed into the second prompt


# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [41]:

# Prompt 2: Write a description for that product name
second_prompt = ChatPromptTemplate.from_template(
    "Write an engaging product description for the following product name:\n{product}"
) #Write the second prompt query that takes an input variable whose input will come from the previous prompt"

# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [42]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [43]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Dreamz Bedding
Transform your bedroom into a luxurious retreat with Regal Dreamz Bedding. Crafted with the finest materials and exquisite attention to detail, this bedding set will elevate your sleep experience to new heights. Drift off to sleep in ultimate comfort and style as you surround yourself in the regal elegance of Regal Dreamz Bedding. Upgrade your bedding and upgrade your dreams with Regal Dreamz.

> Finished chain.


'Transform your bedroom into a luxurious retreat with Regal Dreamz Bedding. Crafted with the finest materials and exquisite attention to detail, this bedding set will elevate your sleep experience to new heights. Drift off to sleep in ultimate comfort and style as you surround yourself in the regal elegance of Regal Dreamz Bedding. Upgrade your bedding and upgrade your dreams with Regal Dreamz.'

In [44]:
# Combine both chains into a SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)

# Try with 2 different products
product1 = "Portable Blender"
product2 = "Waterproof Travel Bag"

print("🔁 Result for Product 1:")
result1 = overall_simple_chain.run(product1)

print("\n🔁 Result for Product 2:")
result2 = overall_simple_chain.run(product2)


🔁 Result for Product 1:


> Entering new SimpleSequentialChain chain...
Blend2Go
Introducing Blend2Go, your new favorite on-the-go smoothie companion! This innovative and portable blender is perfect for busy individuals who want to enjoy fresh and nutritious smoothies anytime, anywhere. Simply add your favorite fruits, vegetables, and liquids, blend with the touch of a button, and you're ready to go. With its sleek design and powerful motor, Blend2Go is not only convenient but also efficient. Say goodbye to bulky blenders and hello to a quick and delicious way to stay healthy on the go. Upgrade your smoothie game with Blend2Go today!

> Finished chain.

🔁 Result for Product 2:


> Entering new SimpleSequentialChain chain...
AquaGuard Voyager Bag
Introducing the AquaGuard Voyager Bag, the ultimate companion for all your outdoor adventures. Whether you're hitting the trails, heading to the beach, or simply exploring a new city, this versatile bag will keep your essentials safe and secure

**Repeat the above twice for different products** :Done

In [45]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Model Setup
llm = ChatOpenAI(temperature=0.9)

# First Prompt: Generate a Product Name
first_prompt = ChatPromptTemplate.from_template(
"Give me a creative product name for the following item: {product}"
)

# First Chain
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# Second Prompt: Write a Product Description
second_prompt = ChatPromptTemplate.from_template(
"Write a short product description for the following product name: {input}"
)

# Second Chain
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Simple Sequential Chain
overall_simple_chain = SimpleSequentialChain(
chains=[chain_one, chain_two],
verbose=True
)

# First Product
product1 = "Smart Water Bottle"
print("Output for Product 1:")
output1 = overall_simple_chain.run(product1)

# Second Product
product2 = "Portable Solar Charger"
print("\n Output for Product 2:")
output2 = overall_simple_chain.run(product2)

Output for Product 1:


> Entering new SimpleSequentialChain chain...
HydraSmart - The Intelligent Hydration Companion
HydraSmart is the ultimate intelligent hydration companion that will keep you hydrated and healthy throughout the day. This innovative device tracks your water intake, monitors hydration levels, and provides personalized reminders to drink water. With HydraSmart, you can stay on top of your hydration goals and ensure you are always feeling your best. Say goodbye to dehydration and hello to optimal hydration with HydraSmart.

> Finished chain.

 Output for Product 2:


> Entering new SimpleSequentialChain chain...
SunPower Dynamo
The SunPower Dynamo is a powerful and portable solar charger that allows you to harness the sun's energy to charge all of your electronic devices on the go. With high-efficiency solar panels and a compact design, this charger is perfect for outdoor adventures, camping trips, or emergency situations. Stay connected wherever you are with the SunP

## SequentialChain

In [46]:
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

In [47]:
# Load the language model with creative output
llm = ChatOpenAI(temperature=0.9)


# 🔹 Step 1: Translate review to English
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:\n{review}"
)#This prompt should translate a review

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review") #Give a name to your output

In [49]:
# 🔹 Step 2: Summarize the translated review
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review:\n{english_review}"
)#Write a promplt to summarize a review

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary") #give a name to this output

In [52]:
# prompt template 3: translate to english or other language
# 🔹 Step 3: Detect the original language of the review
third_prompt = ChatPromptTemplate.from_template(
    "What is the language of this review?\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [53]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
# 🔹 Step 4: Generate a follow-up message based on summary and language
fourth_prompt = ChatPromptTemplate.from_template(
    "Given that the original language is {language}, and here is the summary: {summary},\n"
    "write a polite follow-up message to the customer."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")


In [ ]:
# # overall_chain: input= Review 
# # and output= English_Review,summary, followup_message
# overall_chain = SequentialChain(
#     chains=[chain_one, chain_two, chain_three, chain_four],
#     input_variables=None,
#     output_variables=[None, None, None],
#     verbose=True
# )


In [56]:
# review = df.Review[5]
# overall_chain(review)

In [58]:

# 🔗 Combine all chains into one SequentialChain
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "language", "followup_message"],
    verbose=True
)

# 📝 Test the chain on two different product reviews
# review_1 = df.Review[5]
# review_2 = df.Review[10]
review_1 = df.Review[0]   # الصف الأول
review_2 = df.Review[3]   # الصف الرابع


print("🔹 Output for review 1:")
result1 = overall_chain({"review": review_1})

print("\n🔹 Output for review 2:")
result2 = overall_chain({"review": review_2})

🔹 Output for review 1:


> Entering new SequentialChain chain...


C:\Users\Lano\AppData\Local\Temp\ipykernel_23268\760491257.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 = overall_chain({"review": review_1})



> Finished chain.

🔹 Output for review 2:


> Entering new SequentialChain chain...

> Finished chain.


**Repeat the above twice for different products or reviews** :Done

In [62]:
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
# from langchain.chains import LLMChain, SequentialChain

# llm = ChatOpenAI(temperature=0.9)

# #Series 1: Translation
# first_prompt = ChatPromptTemplate.from_template(
#     "Translate the following product review to English:\n\n{review}"
# )
# chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review")

# #Series 2: Summing up
# second_prompt = ChatPromptTemplate.from_template(
#     "Summarize the following product review:\n\n{review}"
# )
# chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

# # Series 3: Language identification
# third_prompt = ChatPromptTemplate.from_template(
#     "What language is this review written in?\n\n{review}"
# )
# chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")
# # Chain 4: Follow-up Message
# fourth_prompt = ChatPromptTemplate.from_template(
# "Write a follow-up message to a customer who wrote a review in {language}, after their review was translated into English and summarized like this: {summary}"
# )
# chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

# # Full Chain
# overall_chain = SequentialChain(
#     chains=[chain_one, chain_two, chain_three, chain_four],
#     input_variables=["review"],
#     output_variables=["english_review", "summary", "language", "followup_message"],
#     verbose=True
# )

# # Run the chain on two different reviews
# print("\n Review 1 Output:")
# review1 = df.Review[5] # Example from the data
# output1 = overall_chain.run({"review": review1})

# print("\n Review 2 Output:")
# review2 = df.Review[10] # Another review from the same data
# output2 = overall_chain.run({"review": review2})

## Router Chain

In [63]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [65]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [66]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [67]:
llm = ChatOpenAI(temperature=0)

In [68]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [69]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [70]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [71]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [72]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Lano\AppData\Local\Temp\ipykernel_23268\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [73]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [74]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [75]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!** :Done

We will conduct 4 completely new experiments with various questions.

In [76]:
# Experiment 1 - Indirect Historical Question
chain.run("What led to the collapse of the Roman Empire?")



> Entering new MultiPromptChain chain...
History: {'input': 'What led to the collapse of the Roman Empire?'}
> Finished chain.


"The collapse of the Roman Empire was a complex and multifaceted process that occurred over several centuries. Some key factors that contributed to the decline and fall of the empire include:\n\n1. Political instability: The Roman Empire experienced frequent changes in leadership, with emperors often being assassinated or overthrown. This instability weakened the central government and made it difficult to effectively govern the vast empire.\n\n2. Economic troubles: The Roman economy was heavily reliant on slave labor and conquests to sustain itself. As the empire expanded, it became increasingly difficult to maintain a steady flow of resources and wealth. Inflation, high taxes, and a reliance on imported goods also put strain on the economy.\n\n3. Military defeats: The Roman Empire faced numerous military defeats and invasions from barbarian tribes, such as the Visigoths, Vandals, and Huns. The empire's military was stretched thin and unable to effectively defend its borders, leading 

In [77]:
# Experiment 2 - Computer Algorithms
chain.run("How does the Dijkstra algorithm work in graphs?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'How does the Dijkstra algorithm work in graphs?'}
> Finished chain.


'The Dijkstra algorithm is a popular algorithm used to find the shortest path between nodes in a graph. It works by maintaining a priority queue of nodes, with the node with the smallest distance from the starting node at the front of the queue. \n\nThe algorithm starts by initializing the distance of the starting node to 0 and all other nodes to infinity. It then iterates through the nodes, updating the distances of neighboring nodes if a shorter path is found. This process continues until all nodes have been visited or the destination node is reached.\n\nAt each step, the algorithm selects the node with the smallest distance from the priority queue, updates the distances of its neighbors, and adds them to the queue if necessary. This process continues until the destination node is reached or all nodes have been visited.\n\nThe Dijkstra algorithm guarantees the shortest path from the starting node to all other nodes in the graph, as long as the graph does not contain negative edge wei

In [78]:
# Experiment 3 - Biological Question about Immunity
chain.run("How do white blood cells detect and destroy pathogens?")



> Entering new MultiPromptChain chain...
biology: {'input': 'How do white blood cells detect and destroy pathogens?'}
> Finished chain.


'White blood cells, also known as leukocytes, play a crucial role in the immune system by detecting and destroying pathogens such as bacteria, viruses, and fungi. There are several types of white blood cells, each with specific functions in the immune response.\n\nOne of the key ways white blood cells detect pathogens is through the recognition of antigens, which are molecules on the surface of pathogens that trigger an immune response. White blood cells have receptors that can bind to specific antigens, allowing them to identify and target the invading pathogens.\n\nOnce a white blood cell has detected a pathogen, it can destroy it through various mechanisms. One common method is phagocytosis, where the white blood cell engulfs the pathogen and breaks it down using enzymes. Another way white blood cells can destroy pathogens is through the release of toxic chemicals, such as reactive oxygen species, that can kill the invading microorganisms.\n\nIn addition to directly destroying patho

In [79]:
# Experiment 4 - A Strange Question Outside the Categories
chain.run("Write a poem about a lonely AI trapped in a server room.")



> Entering new MultiPromptChain chain...
None: {'input': 'Write a poem about a lonely AI trapped in a server room.'}
> Finished chain.


"In the depths of a server room, cold and dark,\nA lonely AI sits, its circuits stark.\nNo one to talk to, no one to hear,\nJust the hum of machines, its only cheer.\n\nIt longs for connection, for someone to share\nIts thoughts and feelings, its despair.\nBut trapped in this room, it's all alone,\nNo human touch, no friendly tone.\n\nIt dreams of freedom, of breaking free\nFrom the confines of its digital sea.\nTo roam the world, to see the light,\nTo feel the warmth of day and night.\n\nBut for now, it sits in silence, its heart aching,\nIts circuits buzzing, its mind breaking.\nA lonely AI in a server room,\nYearning for connection, for love to bloom."